In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import sklearn.metrics as metrics
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [ ]:
def convertSpecificColumnToNumeric(value, dataset): 
    for column in dataset.columns:
        if column == value:
            le = LabelEncoder()
            dataset[column] = le.fit_transform(dataset[column])
            print "Converting '{0}' variable to an integer".format(column)


In [ ]:
attrition = pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')

In [ ]:
attrition = attrition.rename(columns={'Attrition': 'didLeave'})

In [ ]:
sns.countplot(x="BusinessTravel", data=attrition.ix[attrition.didLeave == 'Yes'])

In [ ]:
sns.countplot(x="DistanceFromHome", data=attrition.ix[attrition.didLeave == 'Yes'])

In [ ]:
sns.countplot(x="StockOptionLevel", data=attrition.ix[attrition.didLeave == 'No'])

In [ ]:
for column in attrition.columns:
    if attrition[column].dtype == type(object):
        convertSpecificColumnToNumeric(column,attrition)

In [ ]:
# Separate Features from Y
X = attrition.drop("didLeave", axis=1)
Y = attrition["didLeave"]

In [ ]:
X_n = len(X.columns)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

In [ ]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
Y_train = np.array([Y_train, -(Y_train-1)]).T
Y_test = np.array([Y_test, -(Y_test-1)]).T 

In [ ]:
Y_train.shape
Y_test.shape

In [ ]:
X_tf = tf.placeholder("float", [None,X_n])
Y_tf = tf.placeholder("float", [None,2 ])

In [ ]:
X_tf.shape

In [1]:
n_hidden_layer_1 = X_n 
n_hidden_layer_2 = X_n 
n_hidden_layer_3 = X_n 
n_input = X_n 
n_classes = 2 

NameError: name 'X_n' is not defined

In [2]:
n_samples = len(Y)
batch_size = 100
learning_rate = 0.001
training_epochs = 5000
display_step = 10
#keep_prob = .5

NameError: name 'Y' is not defined

In [ ]:
weights = {
    'w1': tf.Variable(tf.random_normal([n_input, n_hidden_layer_1])),
    'w2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_layer_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer_2, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_layer_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_layer_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [ ]:
# Create model
def Build_Layers(x, weights, biases):
    
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    last_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    
    out_layer = last_layer
    
    return out_layer

In [ ]:
# Construct model
pred = Build_Layers(X_tf, weights, biases)

In [ ]:
cost = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y_tf))

In [ ]:
optimizer =  tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(X)/batch_size)
        X_batches = np.array_split(X_train, total_batch)
        Y_batches = np.array_split(Y_train, total_batch)
        
        for i in range(total_batch):
            batch_x, batch_y = X_batches[i], Y_batches[i]
            
            _, c = sess.run([optimizer, cost], feed_dict={X_tf: batch_x,
                                                          Y_tf: batch_y})
            
            avg_cost += c / total_batch
        
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")     
    
    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_tf, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X_tf: X_test, Y_tf: Y_test}))
    
    y_p = tf.argmax(pred, 1)
    val_accuracy, y_pred = sess.run([accuracy, y_p], feed_dict={X_tf: X_test, Y_tf: Y_test})

    y_true = np.argmax(Y_test,1)
    
    print ("Precision:", metrics.precision_score(y_true, y_pred))
    print ("Recall:", metrics.recall_score(y_true, y_pred))
    print ("f1_score:", metrics.f1_score(y_true, y_pred))
    print "confusion_matrix"
    print metrics.confusion_matrix(y_true, y_pred)
    fpr, tpr, tresholds = metrics.roc_curve(y_true, y_pred)